In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from torch.optim import AdamW

MODEL_DIR = r"C:\Users\amil\OneDrive\Documents\AI-Driven Personalized Therapy Recommendations system\Module_3\Predictive_model\model\converted_model"
MODEL_BIN_PATH = f"{MODEL_DIR}/pytorch_model.bin"


tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

MODEL_CHECKPOINT = "microsoft/deberta-v3-base"
NUM_LABELS = 19

deberta_model = AutoModel.from_pretrained(MODEL_CHECKPOINT)

# Define your custom classifier
class CustomDebertaClassifier(nn.Module):
    def __init__(self, deberta_model, num_labels):
        super(CustomDebertaClassifier, self).__init__()
        self.deberta = deberta_model  
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_labels)  # 19 output classes
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token representation
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss = self.criterion(logits, labels)  
        return {"loss": loss, "logits": logits}

model = CustomDebertaClassifier(deberta_model, NUM_LABELS)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.load_state_dict(torch.load(MODEL_BIN_PATH, map_location=device))

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

model.eval()
print("✅ Fine-Tuned Custom DeBERTa Model Successfully Loaded from pytorch_model.bin!")


✅ Fine-Tuned Custom DeBERTa Model Successfully Loaded from pytorch_model.bin!


In [2]:
model.eval()

CustomDebertaClassifier(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mental health mapping dictionary
mental_health_mapping = {
    0: "ADHD",
    1: "Anxiety",
    2: "BDD",
    3: "Bipolar",
    4: "BPD",
    5: "Depression",
    6: "Eating Disorder",
    7: "Hoarding Disorder",
    8: "Mental Illness",
    9: "Normal",
    10: "OCD",
    11: "Off My Chest",
    12: "Panic Disorder",
    13: "Personality Disorder",
    14: "PTSD",
    15: "Schizophrenia",
    16: "Social Anxiety",
    17: "Stress",
    18: "Suicidal"
}

def predict(texts):
    """
    Perform inference on a list of input texts.
    Args:
        texts (list): List of input text strings
    Returns:
        list: Predicted labels and probabilities
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Run inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]

    # Convert logits to probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1).cpu().numpy()
    results = []
    for text, pred, prob in zip(texts, preds, probs):
        label = mental_health_mapping.get(pred, "Unknown")  # Map predicted index to label
        confidence = prob.max().item()
        results.append({
            "text": text,
            "prediction": label,   
            "confidence": confidence
        })

    return results

sample_texts = [
    "Imagine waking up in a world where reality feels like a fragile illusion—where your own thoughts betray you, where voices whisper secrets only you can hear, and where the very fabric of existence seems to shift before your eyes. This is the daily experience for someone with schizophrenia, a condition that warps perception, distorts emotions, and disrupts the very nature of self-awareness."
]

predictions = predict(sample_texts)

for result in predictions:
    print(f"Text: {result['text']}")
    print(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2f})")
    print("-" * 50)


Text: Imagine waking up in a world where reality feels like a fragile illusion—where your own thoughts betray you, where voices whisper secrets only you can hear, and where the very fabric of existence seems to shift before your eyes. This is the daily experience for someone with schizophrenia, a condition that warps perception, distorts emotions, and disrupts the very nature of self-awareness.
Prediction: Schizophrenia (Confidence: 1.00)
--------------------------------------------------
